# Respostas as perguntas do Desafio Técnico.

As perguntas para o desafio técnico consistem nas:

1. Qual a média de valor total (total_amount) recebido em um mês considerando todos os yellow táxis da frota?

2. Qual a média de passageiros (passenger\_count) por cada hora do dia que pegaram táxi no mês de maio considerando todos os táxis da
frota?

In [0]:
from pyspark.sql.functions import year, month, avg, round, count, col, hour

In [0]:
class AnalysisTLC:
    def __init__(self):
        self.path_mount_enriched = "/mnt/mount_enriched/"

    def read_files_parquet(self, file_input):
        df = spark.read.parquet(file_input)
        return df

In [0]:

als = AnalysisTLC()

color = 'yellow'
path = als.path_mount_enriched + f"{color}_taxi/metrics/taxi_trip/*.parquet"
df_yellow = als.read_files_parquet(path)

color = 'green'
path = als.path_mount_enriched + f"{color}_taxi/metrics/taxi_trip/*.parquet"
df_green = als.read_files_parquet(path)


In [0]:
df_green.show(10)

+---------+---------------+------------+--------------------+---------------------+
|vendor_id|passenger_count|total_amount|tpep_pickup_datetime|tpep_dropoff_datetime|
+---------+---------------+------------+--------------------+---------------------+
|        2|              1|        16.0| 2023-03-01 00:25:10|  2023-03-01 00:35:47|
|        2|              1|         9.0| 2023-03-01 00:14:29|  2023-03-01 00:25:04|
|        2|              1|       16.68| 2023-02-28 22:59:46|  2023-02-28 23:08:38|
|        2|              1|       25.02| 2023-03-01 00:54:03|  2023-03-01 01:03:14|
|        2|              1|        29.8| 2023-03-01 01:00:09|  2023-03-01 01:14:37|
|        2|              1|       24.24| 2023-03-01 00:09:45|  2023-03-01 00:26:06|
|        2|              5|        37.0| 2023-03-01 00:39:30|  2023-03-01 00:39:33|
|        2|              1|       24.44| 2023-03-01 00:03:07|  2023-03-01 00:14:44|
|        2|              5|       15.84| 2023-03-01 00:42:56|  2023-03-01 00

## Pergunta 1 - Resolução



### Objetivo Principal

Esta query está calculando o valor médio das corridas (total_amount) por mês, considerando apenas os táxis amarelos que finalizaram suas corridas (dropoff) entre janeiro e maio de 2023.

In [0]:
resultado = (
    df_yellow
    .filter(
        (year("tpep_dropoff_datetime") == 2023) & 
        (month("tpep_dropoff_datetime").between(1, 5))
    )
    .groupBy(
        year("tpep_dropoff_datetime").alias("ano"),
        month("tpep_dropoff_datetime").alias("mes")
    )
    .agg(
        round(avg("total_amount"), 2).alias("media_valor_total_mensal")
    )
    .orderBy("ano", "mes")
)


In [0]:
resultado.display()

ano,mes,media_valor_total_mensal
2023,1,27.44
2023,2,27.33
2023,3,28.26
2023,4,28.76
2023,5,29.46


#### Pontos Importantes sobre o código

1. Escolha do tpep_dropoff_datetime:
    - Reflete quando o serviço foi realmente concluído
    - Pode ser mais relevante para análises financeiras (momento do pagamento)

2. Performance: 
    - Filtra primeiro (reduz dados antes do agrupamento)
    - Usa funções nativas do Spark (year, month) otimizadas

3. Escalabilidade: 
    - Funciona bem para grandes volumes de dados típicos em sistemas de táxi1

## Pergunta 2 - Resolução

### Objetivo Principal

Calcular a média de passageiros por hora do dia no mês de maio de 2023.

In [0]:
df = df_yellow.union(df_green)

In [0]:
resultado = (
    df
    .filter(
        (year("tpep_pickup_datetime") == 2023) &
        (month("tpep_pickup_datetime") == 5)
    )
    .withColumn("hora_do_dia", hour("tpep_pickup_datetime"))
    .groupBy("hora_do_dia")
    .agg(
        round(avg("passenger_count"), 2).alias("media_passageiros")
    )

    .orderBy("hora_do_dia")
)

print("Média de passageiros por hora do dia em maio de 2023 para os táxis Yellow:")
display(resultado)

Média de passageiros por hora do dia em maio de 2023 para os táxis Yellow:


hora_do_dia,media_passageiros
0,1.41
1,1.42
2,1.44
3,1.43
4,1.39
5,1.27
6,1.24
7,1.25
8,1.26
9,1.28


#### Pontos Importantes sobre o Código

1. Filtragem de dados: 
    - Foram considerados somente informações as quais corridas se iniciaram no mês de Maio. Para isso foi feito um filtro com a coluna `tpep_pickup_datetime`;

2. Arredondamento de valores decimais para melhor legibilidade;
